In [1]:
import numpy as np
import os
import json
import glob
from moviepy.editor import *
import matplotlib.pyplot as plt
import matplotlib.patches
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation, FFMpegWriter
import matplotlib.animation as animation
import scipy.io as sio
import librosa
import IPython.display as ipd
import librosa.display

In [ ]:
def extract_features(audio_path, sr=48000):
    y, _ = librosa.load(audio_path, sr=sr)

    # Set the hop length for 25 fps
    hop_length = sr // 25

    # Separate harmonics and percussives into two waveforms
    y_harmonic, y_percussive = librosa.effects.hpss(y)

    # Beat track on the percussive signal
    tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)

    # Compute MFCC features from the raw signal
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=16)

    # Compute the first-order differences of the MFCC features
    mfcc_delta = librosa.feature.delta(mfcc)

    # Stack MFCC and MFCC delta features and aggregate between beat events using mean
    beat_mfcc_features = librosa.util.sync(np.vstack([mfcc, mfcc_delta]), beat_frames)

    # Compute chroma features from the harmonic signal
    chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr, hop_length=hop_length)

    # Aggregate chroma features between beat events using median
    beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate=np.median)

    # Finally, stack all beat-synchronous features together
    beat_features = np.vstack([beat_chroma, beat_mfcc_features])
    return beat_features, beat_frames

In [ ]:
audio_path = "Processed/DANCE_C_1.mp3"
beat_features, beat_frames = extract_features(audio_path)

In [ ]:
sr = 48000
audio_path = "Processed/gBR_sBM_c01_d04_mBR0_ch01.mp4"
y, sr = librosa.load(audio_path, sr, offset=1.0, duration=3.0)

# Set the hop length for 25 fps
hop_length = sr // 25

# Separate harmonics and percussives into two waveforms
y_harmonic, y_percussive = librosa.effects.hpss(y)

#display waveform
%matplotlib inline
plt.figure(figsize=(15, 5))
librosa.display.waveplot(y_harmonic, sr=sr, color='b')
ax = plt.gca()
ax.set_axis_off()

In [ ]:
ipd.Audio(y_harmonic, rate=sr)

In [ ]:
beat_frames

In [ ]:
tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)

In [ ]:
first = librosa.time_to_samples(15, sr=sr)
intro = y[:first]
intro_harm = librosa.effects.harmonic(intro)
intro_chroma = librosa.feature.chroma_cqt(intro_harm, sr=sr, hop_length=hop_length)
librosa.display.specshow(intro_chroma, sr=sr, hop_length=hop_length, y_axis='chroma', x_axis='time')
plt.title('Chroma Spectrogram')
plt.colorbar()

In [ ]:
beat_features.shape

In [ ]:
sr=48000

y, _ = librosa.load(audio_path, sr=sr, offset=15.0, duration=12.0)

# Set the hop length for 25 fps
hop_length = sr // 25

# Separate harmonics and percussives into two waveforms
y_harmonic, y_percussive = librosa.effects.hpss(y)

# Beat track on the percussive signal
tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)

# Compute MFCC features from the raw signal
mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=16)

In [ ]:
mfcc.shape

In [ ]:
chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr, hop_length=hop_length

In [ ]:
for music in glob.glob('Music/*.mp3'):
    audio_path = music[6:-4]
    sr = 48000

    y, sr = librosa.load(music, sr)

    # Set the hop length for 25 fps
    hop_length = sr // 25

    # Separate harmonics and percussives into two waveforms
    y_harmonic, y_percussive = librosa.effects.hpss(y)

    h = max(abs(y_harmonic))
    #display waveform
    %matplotlib qt
    fig = plt.figure(figsize=(15, 5))
    librosa.display.waveplot(y_harmonic, sr=sr, color='b')
    ax = plt.gca()
    ax.set_axis_off()

    patch = matplotlib.patches.Rectangle((0, -h), width=0, height=2*h, fc=[1,1,1,0.5])
    ax.add_patch(patch)

    def animate(i):
        patch.set_width(i/25)
        return patch,

    anim = animation.FuncAnimation(fig, animate,
                                   frames=250,
                                   interval=40,
                                   blit=True)
    FFwriter = animation.FFMpegWriter(fps=25)
    anim.save('Music/{}.mp4'.format(audio_path), writer=FFwriter, dpi=300)
#     plt.show()

In [ ]:
for file in ['Waltz_Gen', 'ChaCha_Gen']:
    music = f'Test/Video/{file}.mp3'

    sr = 48000

    y, sr = librosa.load(music, sr, offset=0.0, duration=12.0)

    # Set the hop length for 25 fps
    hop_length = sr // 25

    # Separate harmonics and percussives into two waveforms
    y_harmonic, y_percussive = librosa.effects.hpss(y)

    h = max(abs(y_harmonic))
    #display waveform
    %matplotlib qt
    fig = plt.figure(figsize=(15, 5))
    
    librosa.display.waveplot(y_harmonic, sr=sr, color=[0.9,0.7,0.7])
    ax = plt.gca()
    ax.set_axis_off()

    patch = matplotlib.patches.Rectangle((0, -h), width=0, height=2*h, fc=[1,1,1,0.5])
    ax.add_patch(patch)

    def animate(i):
        patch.set_width(i/25)
        return patch,

    anim = animation.FuncAnimation(fig, animate,
                                   frames=300,
                                   interval=40,
                                   blit=True)
    FFwriter = animation.FFMpegWriter(fps=25)
    anim.save(f'Test/Video/{file}.mp4', writer=FFwriter, dpi=100)

In [ ]:
os.system('ffmpeg -y -i ChaCha_Gen.wav -ss 00:00:02.5 -t 00:00:12.0 -vn -ar 44100 -ac 2 -b:a 192k Test/Video/ChaCha_Gen.mp3')
os.system('ffmpeg -y -i ChaCha.mp3 -ss 00:00:02.5 -t 00:00:12.0 -vn -ar 44100 -ac 2 -b:a 192k Test/Video/ChaCha.mp3')

In [ ]:
os.system('ffmpeg -y -i Waltz_Gen.wav -ss 00:00:03 -t 00:00:12.0 -vn -ar 44100 -ac 2 -b:a 192k Test/Video/Waltz_Gen.mp3')
os.system('ffmpeg -y -i Waltz.mp3 -ss 00:00:03 -t 00:00:12.0 -vn -ar 44100 -ac 2 -b:a 192k Test/Video/Waltz.mp3')
